# Introduction to Data Science 2025

# Week 4

In this week's exercise, we look at prompting and zero- and few-shot task settings. Below is a text generation example from https://github.com/TurkuNLP/intro-to-nlp/blob/master/text_generation_pipeline_example.ipynb demonstrating how to load a text generation pipeline with a pre-trained model and generate text with a given prompt. Your task is to load a similar pre-trained generative model and assess whether the model succeeds at a set of tasks in zero-shot, one-shot, and two-shot settings.

**Note: Downloading and running the pre-trained model locally may take some time. Alternatively, you can open and run this notebook on [Google Colab](https://colab.research.google.com/), as assumed in the following example.**

## Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [1]:
!pip install --quiet transformers

^C



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

c:\koodit\DataScienceMasters\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models. 

In [2]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

c:\koodit\DataScienceMasters\env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tuoma\.cache\huggingface\hub\models--TurkuNLP--gpt3-finnish-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTT

Instantiate a text generation pipeline using the tokenizer and model.

In [ ]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

Device set to use cpu


We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [4]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

[{'generated_text': 'Terve, miten menee?”\n”Hyvin.”\n”Entä te?”\n”Oikein hyvin.”\n”Näytät aika kalpealta.'}]


Just print the text

In [5]:
print(output[0]['generated_text'])

Terve, miten menee?”
”Hyvin.”
”Entä te?”
”Oikein hyvin.”
”Näytät aika kalpealta.


We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [ ]:
output = pipe(
    'Terve, miten menee?',
    do_sample=True,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
 Mua vähän ärsyttää tämä asia, koska olen itse herkkä aistimaan ihmisten tunnetiloja ja olen itsekin huomannut, että ihmiset usein kertovat suoraan jos jokin on pielessä. Mutta välillä


## Exercise 1

Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and two-shot settings:

- binary sentiment classification (positive / negative)

- person name recognition

- two-digit addition (e.g. 11 + 22 = 33)

For example, for assessing whether a generative model can name capital cities, we could use the following prompts:

- zero-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- one-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- two-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Denmark?\
	>Answer: Copenhagen
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""

You can do the tasks either in English or Finnish and use a generative model of your choice from the Hugging Face models repository, for example the following models:

- English: `gpt2-large`
- Finnish: `TurkuNLP/gpt3-finnish-large`

You can either come up with your own instructions for the tasks or use the following:

- English:
	- binary sentiment classification: "Do the following texts express a positive or negative sentiment?"
	- person name recognition: "List the person names occurring in the following texts."
	- two-digit addition: "This is a first grade math exam."
- Finnish:
	- binary sentiment classification: "Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?"
	- person name recognition: "Listaa seuraavissa teksteissä mainitut henkilönnimet."
	- two-digit addition: "Tämä on ensimmäisen luokan matematiikan koe."

Come up with at least two test cases for each of the three tasks, and come up with your own one- and two-shot examples.

In [67]:
### Sentiment Classification

def get_evaluation(prompt, shot, ind, answer, should, tokens=5):
    print(f"{shot}-shot {ind}. example.\n\tShould be: {should}\n")
    generated = pipe(prompt, max_new_tokens=tokens)[0]["generated_text"]
    print(generated)
    print("-" * 20 + "\n")

# Zero-shot:

prompt = """Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Tämä on todella hyvä elokuva!
Vastaus:"""
get_evaluation(prompt, "Zero", 1, "Vastaus: ", "Positiivinen")

Zero-shot 1. example.
	Should be: Positiivinen

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Tämä on todella hyvä elokuva!
Vastaus: Kyllä, tämä on todella
--------------------



**Submit this exercise by submitting your code and your answers to the above questions as comments on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**

In [57]:
# One shot:

prompt = """Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Tämä on todella hyvä elokuva!
Vastaus: Positiivinen

Teksti: En nauti kahvin juomisesta iltapäivällä, se häiritsee yöunia.
Vastaus:"""
get_evaluation(prompt, "One", 1, "Vastaus: ", "Negatiivinen")
prompt = """Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Minusta kermakakut mansikoilla ovat aivan parhaita.
Vastaus: Positiivinen

Teksti: En nauti kahvin juomisesta iltapäivällä, se häiritsee yöunia.
Vastaus:"""
get_evaluation(prompt, "One", 2, "Vastaus: ", "Negatiivinen")



One-shot 1. example.
	Should be: Negatiivinen

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Tämä on todella hyvä elokuva!
Vastaus: Positiivinen

Teksti: En nauti kahvin juomisesta iltapäivällä, se häiritsee yöunia.
Vastaus: Kahvin juominen iltapäivällä ei häiritse
--------------------

One-shot 2. example.
	Should be: Negatiivinen

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Minusta kermakakut mansikoilla ovat aivan parhaita.
Vastaus: Positiivinen

Teksti: En nauti kahvin juomisesta iltapäivällä, se häiritsee yöunia.
Vastaus: Ei haittaa, sillä illalla
--------------------



In [58]:
# Two-shot:

prompt = """Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Tämä on todella hyvä elokuva!
Vastaus: Positiivinen

Teksti: En nauti kahvin juomisesta iltapäivällä, se häiritsee yöunia.
Vastaus: Negatiivinen

Teksti: Haluan aina syödä tivolissa hattaraa, se on makeaa ja sulaa suuhun.
Vastaus:"""

get_evaluation(prompt, "Two", 1, "Vastaus: ", "Positiivinen")

prompt = """Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Minusta kermakakut mansikoilla ovat aivan parhaita.
Vastaus: Positiivinen

Teksti: En nauti kahvin juomisesta iltapäivällä, se häiritsee yöunia.
Vastaus: Negatiivinen

Teksti: Saisi jo tulla kesä, alkaa käymään hermoille nämä loskakelit
Vastaus:"""

get_evaluation(prompt, "Two", 2, "Vastaus: ", "Negatiivinen")


Two-shot 1. example.
	Should be: Positiivinen

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Tämä on todella hyvä elokuva!
Vastaus: Positiivinen

Teksti: En nauti kahvin juomisesta iltapäivällä, se häiritsee yöunia.
Vastaus: Negatiivinen

Teksti: Haluan aina syödä tivolissa hattaraa, se on makeaa ja sulaa suuhun.
Vastaus: Negatiivinen

Teksti: Haluan
--------------------

Two-shot 2. example.
	Should be: Negatiivinen

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Minusta kermakakut mansikoilla ovat aivan parhaita.
Vastaus: Positiivinen

Teksti: En nauti kahvin juomisesta iltapäivällä, se häiritsee yöunia.
Vastaus: Negatiivinen

Teksti: Saisi jo tulla kesä, alkaa käymään hermoille nämä loskakelit
Vastaus: Negatiivinen

Teksti: Haluan
--------------------



In [99]:
## Name recognition

# Zero-shot

prompt = """Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus:"""

get_evaluation(prompt, "Zero", 1, "Vastaus: ", "Toni, Joonas ja Maija")

Zero-shot 1. example.
	Should be: Toni, Joonas ja Maija

Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus:
Oikein sopiva nimi!
--------------------



In [100]:
# One-shot:

prompt = """Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus:Toni, Joonas ja Maija

Teksti: Apostoli Tuomaksen ensimmäinen tehtävä oli muistuttaa muita heidän kärsimättömyydestään, varsinkin Anttia
Vastaus:"""

get_evaluation(prompt, "One", 1, "Vastaus: ", "Tuomas ja Antti")

prompt = """Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus: Toni, Joonas ja Maija

Teksti: Tapasin isovanhempani ensimmäistä kertaa kymmenen vuotiaana. Kauko papasta ja Pirkko mummosta tuli minulle välittömästi läheisiä ihmisiä.
Vastaus:"""

get_evaluation(prompt, "One", 2, "Vastaus: ", "Kauko ja Pirkko")

One-shot 1. example.
	Should be: Tuomas ja Antti

Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus:Toni, Joonas ja Maija

Teksti: Apostoli Tuomaksen ensimmäinen tehtävä oli muistuttaa muita heidän kärsimättömyydestään, varsinkin Anttia
Vastaus:Apostoli Tuomaksen tehtävä
--------------------

One-shot 2. example.
	Should be: Kauko ja Pirkko

Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus: Toni, Joonas ja Maija

Teksti: Tapasin isovanhempani ensimmäistä kertaa kymmenen vuotiaana. Kauko papasta ja Pirkko mummosta tuli minulle välittömästi läheisiä ihmisiä.
Vastaus: Muistan kun mummoni kertoi minulle
--------------------



In [101]:
# Two-shot:

prompt = """Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus:Toni, Joonas ja Maija

Teksti: Apostoli Tuomaksen ensimmäinen tehtävä oli muistuttaa muita heidän kärsimättömyydestään, varsinkin Anttia
Vastaus: Tuomas ja Antti

Teksti: Harkitsimme pitkään uudelle koiraa annettavaa nimeä. Siskoni ehdotti Jesseä, minä Kimmoa ja äitimme Heikkiä
Vastaus:"""

get_evaluation(prompt, "Two", 1, "Vastaus: ", "Jesse, Kimmo ja Heikki")

prompt = """Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus: Toni, Joonas ja Maija

Teksti: Tapasin isovanhempani ensimmäistä kertaa kymmenen vuotiaana. Kauko papasta ja Pirkko mummosta tuli minulle välittömästi läheisiä ihmisiä.
Vastaus: Kauko ja Pirkko

Teksti: Martti tuli maailmalla tunnetuksi diplomaatin taidoistaan, toisin kuin Sauli, joka tunnettiin kansanedustajana.
Vastaus:"""

get_evaluation(prompt, "Two", 2, "Vastaus: ", "Martti ja Sauli")

Two-shot 1. example.
	Should be: Jesse, Kimmo ja Heikki

Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus:Toni, Joonas ja Maija

Teksti: Apostoli Tuomaksen ensimmäinen tehtävä oli muistuttaa muita heidän kärsimättömyydestään, varsinkin Anttia
Vastaus: Tuomas ja Antti

Teksti: Harkitsimme pitkään uudelle koiraa annettavaa nimeä. Siskoni ehdotti Jesseä, minä Kimmoa ja äitimme Heikkiä
Vastaus: Jesse

Teksti: Hän oli
--------------------

Two-shot 2. example.
	Should be: Martti ja Sauli

Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Toni ja Joonas kävivät tänään Maijalla kylässä
Vastaus: Toni, Joonas ja Maija

Teksti: Tapasin isovanhempani ensimmäistä kertaa kymmenen vuotiaana. Kauko papasta ja Pirkko mummosta tuli minulle välittömästi läheisiä ihmisiä.
Vastaus: Kauko ja Pirkko

Teksti: Martti tuli maailmalla tunnetuksi diplomaatin taidoistaan, toisin kuin Sauli, joka tunnettiin kansanedustajana.
Vastaus: S

In [102]:
### Additive operation

# Zero-shot

prompt = """Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 3 + 4
Vastaus: """
get_evaluation(prompt, "Zero", 1, "Vastaus: ", "7", 2)


Zero-shot 1. example.
	Should be: 7

Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 3 + 4
Vastaus: =5

--------------------



In [103]:
# One-shot

prompt = """Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 3 + 4
Vastaus: 7

Kysymys: 2 + 12
Vastaus: """
get_evaluation(prompt, "One", 1, "Vastaus: ", "14", 2)

prompt = """Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 5 + 2
Vastaus: 7

Kysymys: 1 + 1
Vastaus: """
get_evaluation(prompt, "One", 2, "Vastaus: ", "2", 2)

One-shot 1. example.
	Should be: 14

Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 3 + 4
Vastaus: 7

Kysymys: 2 + 12
Vastaus: .....
--------------------

One-shot 2. example.
	Should be: 2

Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 5 + 2
Vastaus: 7

Kysymys: 1 + 1
Vastaus: ______
--------------------



In [98]:
# Two-shot

prompt = """Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 3 + 4
Vastaus: 7

Kysymys: 2 + 12
Vastaus: 14

Kysymys: 1 + 2
Vastaus: """
get_evaluation(prompt, "Two", 1, "Vastaus: ", "3", 2)

prompt = """Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 5 + 2
Vastaus: 7

Kysymys: 4 + 5
Vastaus: 9

Kysymys: 3 + 3
Vastaus: """
get_evaluation(prompt, "Two", 1, "Vastaus: ", "6", 2)

Two-shot 1. example.
	Should be: 3

Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 3 + 4
Vastaus: 7

Kysymys: 2 + 12
Vastaus: 14

Kysymys: 1 + 2
Vastaus: _____
--------------------

Two-shot 1. example.
	Should be: 6

Tämä on ensimmäisen luokan matematiikan koe
Kysymys: 5 + 2
Vastaus: 7

Kysymys: 4 + 5
Vastaus: 9

Kysymys: 3 + 3
Vastaus:  4


--------------------

